The Llama 2 is a collection of pretrained and fine-tuned generative text models, ranging from 7 billion to 70 billion parameters, designed for dialogue use cases.

 It outperforms open-source chat models on most benchmarks and is on par with popular closed-source models in human evaluations for helpfulness and safety.

[Llama 2 13B-chat](https://huggingface.co/meta-llama/Llama-2-13b-chat)

[link text](https://)`llama.cpp`'s objective is to run the LLaMA model with 4-bit integer quantization on MacBook. It is a plain C/C++ implementation optimized for Apple silicon and x86 architectures, supporting various integer quantization and BLAS libraries. Originally a web chat example, it now serves as a development playground for ggml library features.

`GGML`, a C library for machine learning, facilitates the distribution of large language models (LLMs). It utilizes quantization to enable efficient LLM execution on consumer hardware. GGML files contain binary-encoded data, including version number, hyperparameters, vocabulary, and weights. The vocabulary comprises tokens for language generation, while the weights determine the LLM's size. Quantization reduces precision to optimize resource usage.

#  Quantized Models from the Hugging Face Community

The Hugging Face community provides quantized models, which allow us to efficiently and effectively utilize the model on the T4 GPU. It is important to consult reliable sources before using any model.

There are several variations available, but the ones that interest us are based on the GGLM library.

We can see the different variations that Llama-2-13B-GGML has [here](https://huggingface.co/models?search=llama%202%20ggml).



In this case, we will use the model called [Llama-2-13B-chat-GGML](https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML).

#**step 1 Install All Required Packages**

In [4]:
!nvidia-smi

Fri Jun 14 05:16:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.5 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.4/863.4 kB 8.7 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 10.1 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 54.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 34.0 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 7.0 MB/s eta 0:00:00
    Using cached wheel-0.43.0-py3-none-any.whl (65 kB)
    Creating /tmp/pip-build-env-wssef1jn/overlay/local/bin
    changing mode of /tmp/pip-build-env-wssef1jn/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-wssef1jn/overlay/local/bin/wheel to 755
    c

In [6]:
#model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
#model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

model_name_or_path = "TheBloke/Llama-2-7B-Chat-GGML"
model_basename = "llama-2-7b-chat.ggmlv3.q8_0.bin" # the model is in bin format

#**step 2 Import All the Required Libraries**

In [7]:
from huggingface_hub import hf_hub_download

In [8]:
from llama_cpp import Llama

#**step 3 Download the Model**


In [9]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-7b-chat.ggmlv3.q8_0.bin:   0%|          | 0.00/7.16G [00:00<?, ?B/s]

In [10]:
model_path

'/root/.cache/huggingface/hub/models--TheBloke--Llama-2-7B-Chat-GGML/snapshots/76cd63c351ae389e1d4b91cab2cf470aab11864b/llama-2-7b-chat.ggmlv3.q8_0.bin'

#**step4  Loading the Model**

In [39]:
# GPU # adjust for speed
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [40]:
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

32

#**step 5 Create a Prompt Template**

In [41]:
prompt = "Write a linear regression code"
prompt_template=f'''SYSTEM: You are a helpful , respectful and honest assistant. Always answer as helpfully.

USER: {prompt}

ASSISTANT:
'''

#**step 6  Generating the Response**

In [42]:
response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.1, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

In [43]:
print(response)

{'id': 'cmpl-37274c57-2ed6-4b72-abff-263995432fa9', 'object': 'text_completion', 'created': 1718343529, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-7B-Chat-GGML/snapshots/76cd63c351ae389e1d4b91cab2cf470aab11864b/llama-2-7b-chat.ggmlv3.q8_0.bin', 'choices': [{'text': 'SYSTEM: You are a helpful , respectful and honest assistant. Always answer as helpfully.\n\nUSER: Write a linear regression code\n\nASSISTANT:\nOf course! Here is an example of how to write a simple linear regression code in Python using scikit-learn library:\n```\nimport numpy as np\nfrom sklearn import datasets\nfrom sklearn.linear_model import LinearRegression\nfrom sklearn.metrics import mean_squared_error, r2_score\n\n# Load the Boston Housing dataset (available in scikit-learn)\nboston = datasets.load_boston()\nX = boston.data[:, :2]  # we only take the first two features.\ny = boston.target\n\n# Create a LinearRegression object and fit the data\nreg = LinearRegression().fit(X, y)\n\n# Print the 

In [44]:
print(response["choices"][0]["text"])

SYSTEM: You are a helpful , respectful and honest assistant. Always answer as helpfully.

USER: Write a linear regression code

ASSISTANT:
Of course! Here is an example of how to write a simple linear regression code in Python using scikit-learn library:
```
import numpy as np
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Load the Boston Housing dataset (available in scikit-learn)
boston = datasets.load_boston()
X = boston.data[:, :2]  # we only take the first two features.
y = boston.target

# Create a LinearRegression object and fit the data
reg = LinearRegression().fit(X, y)

# Print the coefficients
print("Coefficients:", reg.coef_)

# Predict values for new input data
new_data = np.random.randn(5, 2)
predictions = reg.predict(new_data)

# Evaluate the model using mean squared error and R-squared score
mse = mean_squared_error(y, predictions)
r2 = r2_score(y


#Experiment